In [1]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import KFold
from numpy.random import rand
from sklearn.neighbors import KNeighborsClassifier
import matplotlib.pyplot as plt

In [2]:
def error_rate(features, target, x, opts):
    # parameters
    k     = opts['k']
    cv    = opts['cv']

    kf = KFold(n_splits=cv, shuffle=True, random_state=2)

    total_error = 0

    for train_index, test_index in kf.split(features):
      X_train, X_test = features[train_index], features[test_index]
      y_train, y_test = target[train_index], target[test_index]

      # Number of instances
      num_train = np.size(X_train, 0)
      num_test  = np.size(X_test, 0)
      
      # Define selected features
      xtrain = X_train[:, x==1]
      ytrain = y_train.reshape(num_train)
      xtest  = X_test[:, x==1]
      ytest  = y_test.reshape(num_test)

      # Training
      knn = KNeighborsClassifier(n_neighbors=k)
      knn.fit(xtrain, ytrain)

      # Prediction
      ypred = knn.predict(xtest)
      acc   = np.sum(ytest == ypred) / num_test
      error = 1 - acc

      total_error = total_error + error
      
    return total_error/cv

In [3]:
def Fun(features, target, x, opts):
    # Parameters
    alpha    = 0.99
    beta     = 1 - alpha
    # Original feature size
    max_feat = len(x)
    # Number of selected features
    num_feat = np.sum(x == 1)
    # Solve if no feature selected
    if num_feat == 0:
        cost  = 1
    else:
        # Get error rate
        error = error_rate(features, target, x, opts)
        # Objective function
        cost  = alpha * error + beta * (num_feat / max_feat)
        
    return cost

In [4]:
def init_position(lb, ub, N, dim):
    X = np.zeros([N, dim], dtype='float')
    for i in range(N):
        for d in range(dim):
            X[i,d] = lb[0,d] + (ub[0,d] - lb[0,d]) * rand()        
    
    return X

In [5]:
def init_velocity(lb, ub, N, dim):
    V    = np.zeros([N, dim], dtype='float')
    Vmax = np.zeros([1, dim], dtype='float')
    Vmin = np.zeros([1, dim], dtype='float')
    # Maximum & minimum velocity
    for d in range(dim):
        Vmax[0,d] = (ub[0,d] - lb[0,d]) / 2
        Vmin[0,d] = -Vmax[0,d]
        
    for i in range(N):
        for d in range(dim):
            V[i,d] = Vmin[0,d] + (Vmax[0,d] - Vmin[0,d]) * rand()
        
    return V, Vmax, Vmin


In [6]:
def binary_conversion(X, thres, N, dim):
    Xbin = np.zeros([N, dim], dtype='int')
    for i in range(N):
        for d in range(dim):
            xi = 1/(1 + np.exp(-X[i,d]))
            # if xi > thres:
            if X[i,d] > 0.5:
                Xbin[i,d] = 1
            else:
                Xbin[i,d] = 0
    
    return Xbin

In [7]:
def boundary(x, lb, ub):
    if x < lb:
        x = lb
    if x > ub:
        x = ub
    
    return x

In [8]:
def pso(features, target, opts):
    # Parameters
    ub    = 1
    lb    = 0
    thres = rand()    # NEED TO CHANGE
    w     = 0.9       # inertia weight
    c1    = 2         # acceleration factor
    c2    = 2         # acceleration factor
    
    print(thres)

    N        = opts['N']
    max_iter = opts['T']
    if 'w' in opts:
        w    = opts['w']
    if 'c1' in opts:
        c1   = opts['c1']
    if 'c2' in opts:
        c2   = opts['c2'] 
    
    # Dimension
    dim = np.size(features, 1)
    if np.size(lb) == 1:
        ub = ub * np.ones([1, dim], dtype='float')
        lb = lb * np.ones([1, dim], dtype='float')
        
    # Initialize position & velocity
    X             = init_position(lb, ub, N, dim)
    V, Vmax, Vmin = init_velocity(lb, ub, N, dim) 
    
    # Pre
    fit   = np.zeros([N, 1], dtype='float')
    Xgb   = np.zeros([1, dim], dtype='float')
    fitG  = float('inf')
    Xpb   = np.zeros([N, dim], dtype='float')
    fitP  = float('inf') * np.ones([N, 1], dtype='float')
    curve = np.zeros([1, max_iter], dtype='float') 
    t     = 0
    
    while t < max_iter:
        # Binary conversion
        Xbin = binary_conversion(X, thres, N, dim)
        
        # Fitness
        for i in range(N):
            fit[i,0] = Fun(features, target, Xbin[i,:], opts)
            if fit[i,0] < fitP[i,0]:
                Xpb[i,:]  = X[i,:]
                fitP[i,0] = fit[i,0]
            if fitP[i,0] < fitG:
                Xgb[0,:]  = Xpb[i,:]
                fitG      = fitP[i,0]
        
        # Store result
        curve[0,t] = fitG.copy()
        print("Iteration:", t + 1)
        print("Best (PSO):", curve[0,t])
        t += 1
        
        for i in range(N):
            for d in range(dim):
                # Update velocity
                r1     = rand()
                r2     = rand()
                V[i,d] = w * V[i,d] + c1 * r1 * (Xpb[i,d] - X[i,d]) + c2 * r2 * (Xgb[0,d] - X[i,d]) 
                # Boundary
                V[i,d] = boundary(V[i,d], Vmin[0,d], Vmax[0,d])
                # Update position
                X[i,d] = X[i,d] + V[i,d]
                # Boundary
                X[i,d] = boundary(X[i,d], lb[0,d], ub[0,d])
    
                
    # Best feature subset
    Gbin       = binary_conversion(Xgb, thres, 1, dim) 
    Gbin       = Gbin.reshape(dim)
    pos        = np.asarray(range(0, dim))    
    sel_index  = pos[Gbin == 1]
    num_feat   = len(sel_index)
    # Create dictionary
    pso_data = {'sf': sel_index, 'c': curve, 'nf': num_feat}
    
    return pso_data

In [9]:
column_headers = ['Type', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols',
                  'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color Intensity',
                   'Hue', 'OD280/OD315 of diluted wines', 'Proline']

In [10]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None, names=column_headers)
data = data.values
data

array([[1.000e+00, 1.423e+01, 1.710e+00, ..., 1.040e+00, 3.920e+00,
        1.065e+03],
       [1.000e+00, 1.320e+01, 1.780e+00, ..., 1.050e+00, 3.400e+00,
        1.050e+03],
       [1.000e+00, 1.316e+01, 2.360e+00, ..., 1.030e+00, 3.170e+00,
        1.185e+03],
       ...,
       [3.000e+00, 1.327e+01, 4.280e+00, ..., 5.900e-01, 1.560e+00,
        8.350e+02],
       [3.000e+00, 1.317e+01, 2.590e+00, ..., 6.000e-01, 1.620e+00,
        8.400e+02],
       [3.000e+00, 1.413e+01, 4.100e+00, ..., 6.100e-01, 1.600e+00,
        5.600e+02]])

In [11]:
features  = np.asarray(data[:, 1:])
target = np.asarray(data[:, 0])

In [12]:
num_runs = 20    # number of independent runs
k        = 5     # k-value in KNN
N        = 5     # number of particles
T        = 20    # maximum number of iterations
cv       = 10    # K-fold cross-validation
opts     = {'k':k, 'N':N, 'T':T, 'cv': cv}
dim      = features.shape[1]

In [13]:
acc_arr          = []
feature_size_arr = []
run_count        = 0

while run_count < num_runs:
  run_count += 1
  print("Run ", run_count)
  print("-----------------------------------")
  fmdl = pso(features, target, opts)
  sf   = fmdl['sf']
  
  if sf.size == 0:
    sf = np.arange(dim)
    
  print("SF", sf)

  kf = KFold(n_splits=cv, shuffle=True, random_state=2)

  total_Acc = 0

  for train_index, test_index in kf.split(features):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = target[train_index], target[test_index]

    # Number of instances
    num_train = np.size(X_train, 0)
    num_test  = np.size(X_test, 0)
    
    # Define selected features
    xtrain = X_train[:, sf]
    ytrain = y_train.reshape(num_train)
    xtest  = X_test[:, sf]
    ytest  = y_test.reshape(num_test)

    # Training
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(xtrain, ytrain)

    # Prediction
    ypred = knn.predict(xtest)
    acc   = np.sum(ytest == ypred) / num_test

    total_Acc = total_Acc + acc

  Accuracy = 100 * (total_Acc/cv)
  print("Accuracy:", Accuracy)
  acc_arr.append(Accuracy)

  num_feat = fmdl['nf']
  print("Feature Size:", num_feat)
  feature_size_arr.append(num_feat)
  print("--------------------------------------------------")

Run  1
-----------------------------------
0.9024242009632556
Iteration: 1
Best (PSO): 0.299472850678733
Iteration: 2
Best (PSO): 0.2987036199095023
Iteration: 3
Best (PSO): 0.2987036199095023
Iteration: 4
Best (PSO): 0.25328733031674217
Iteration: 5
Best (PSO): 0.15266968325791855
Iteration: 6
Best (PSO): 0.07643891402714936
Iteration: 7
Best (PSO): 0.07170814479638012
Iteration: 8
Best (PSO): 0.04311538461538463
Iteration: 9
Best (PSO): 0.04311538461538463
Iteration: 10
Best (PSO): 0.04311538461538463
Iteration: 11
Best (PSO): 0.04311538461538463
Iteration: 12
Best (PSO): 0.04311538461538463
Iteration: 13
Best (PSO): 0.04311538461538463
Iteration: 14
Best (PSO): 0.04311538461538463
Iteration: 15
Best (PSO): 0.04311538461538463
Iteration: 16
Best (PSO): 0.04311538461538463
Iteration: 17
Best (PSO): 0.04311538461538463
Iteration: 18
Best (PSO): 0.04311538461538463
Iteration: 19
Best (PSO): 0.04311538461538463
Iteration: 20
Best (PSO): 0.04311538461538463
SF [ 0  2  5  6  8 10]
Accuracy

In [14]:
print("Average Accuracy: ", np.mean(acc_arr))
print("Average number of features selected: ", np.mean(feature_size_arr))

Average Accuracy:  92.13235294117646
Average number of features selected:  5.35
